In [ ]:
import numpy
import surprise
print(f"Numpy version: {numpy.__version__}")
print(f"Surprise version: {surprise.__version__}")

# --- Importaciones ---
import pandas as pd
# CAMBIO 1: Importamos SVD en lugar de KNNBasic
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import os
import mlflow
import mlflow.sklearn

print("Librerías listas, incluyendo MLflow y SVD.")

# --- Carga de Datos (Sin cambios, pero ahora entendemos los límites) ---
try:
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

DATA_DIR = os.path.join(BASE_DIR, 'data', 'ml-25m')
RATINGS_PATH = os.path.join(DATA_DIR, 'ratings.csv')
ratings_df = pd.read_csv(RATINGS_PATH)

# Mantenemos nuestro muestreo. SVD lo manejará sin problemas.
n_users = 40000
n_movies = 20000
user_ids = ratings_df['userId'].value_counts().nlargest(n_users).index
movie_ids = ratings_df['movieId'].value_counts().nlargest(n_movies).index
sampled_df = ratings_df[(ratings_df['userId'].isin(user_ids)) & (ratings_df['movieId'].isin(movie_ids))]

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(sampled_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# CAMBIO 2: Definimos un nuevo experimento para SVD para mantener todo ordenado.
mlflow.set_experiment("LatentLens-SVD-Evaluation")

with mlflow.start_run() as run:
    print(f"MLflow Run ID: {run.info.run_id}")
    
    # --- CAMBIO 3: Parámetros específicos para SVD ---
    # En lugar de K, SVD tiene "factores latentes". Son las dimensiones del "espacio de gustos".
    n_factors = 100
    n_epochs = 20 # Cuántas veces el algoritmo mira los datos para aprender.
    
    # Registramos los nuevos hiperparámetros.
    mlflow.log_param("model_type", "SVD")
    mlflow.log_param("n_factors", n_factors)
    mlflow.log_param("n_epochs", n_epochs)
    mlflow.log_param("user_sample_size", n_users)
    mlflow.log_param("movie_sample_size", n_movies)
    
    # --- CAMBIO 4: Instanciamos y entrenamos el modelo SVD ---
    # Es un reemplazo directo de KNNBasic.
    model = SVD(n_factors=n_factors, n_epochs=n_epochs, random_state=42)
    
    print("Entrenando el modelo SVD... (Esto es computacionalmente intensivo, pero eficiente en memoria)")
    model.fit(trainset)
    print("¡Modelo SVD entrenado!")
    
    # --- Evaluación (sin cambios en el proceso) ---
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    
    mlflow.log_metric("rmse", rmse)

    # --- CAMBIO 5: Registramos el modelo SVD ---
    # Le damos un nombre diferente al artefacto para no confundirlo.
    mlflow.sklearn.log_model(model, "surprise_svd_model")
    print("Modelo SVD (artefacto) y métricas registradas en MLflow.")

print("\n¡Ejecución de MLflow con SVD finalizada! Revisa la UI.")

1.26.4
1.1.4
Librerías listas, incluyendo MLflow.


2025/08/07 17:01:04 INFO mlflow.tracking.fluent: Experiment with name 'LatentLens-KNN-Evaluation' does not exist. Creating a new experiment.


MLflow Run ID: cad377c367094d5883d732d70a77fe2e
Computing the cosine similarity matrix...


MemoryError: Unable to allocate 2.98 GiB for an array with shape (20000, 20000) and data type float64